In [8]:
"""
前処理の参考
https://qiita.com/Hironsan/items/2466fe0f344115aff177
https://torasenriwohashiru.hatenadiary.org/entry/20110806/1312558290
"""
import pandas as pd
import re
import nltk

def preprocessing(txt_file):#txt_file：前処理対象のファイル
    df_origin = pd.read_csv(txt_file,sep="\t")
    df_prep = df_origin
    for num in range (len(df_origin)):
        text = df_origin["TITLE"][num]#TITLEだけを前処理対象にする
        t_prep = re.sub(re.compile('<.*?>'), '',text)#htmlタグ削除
        t_prep = re.sub(r'[\"\'.,:;\(\)#\|\*\+\!\?#$%&/\]\[\{\}]', '', t_prep)#記号削除
        t_prep = re.sub('[0-9]+', '', t_prep)#数字を削除
        t_prep = re.sub(r"-","",t_prep)#ハイフン削除
        t_prep = t_prep.lower()#小文字化
        t_prep = " ".join([nltk.PorterStemmer().stem(t) for t in t_prep.split()])#ステミング(語幹の抽出)
        df_prep["TITLE"][num] = t_prep#処理後の文字列に置き換え

    return df_prep

#学習データ，検証データ，評価データそれぞれに前処理をする
train_prep = preprocessing("train.txt")
valid_prep = preprocessing("valid.txt")
test_prep = preprocessing("test.txt")

In [9]:
len(train_prep)

10672

In [37]:
"""
tf-idf実装の参考
https://qiita.com/tsugar/items/0391c9a45842f9d9ae69

tf-idf:文章中に含まれる単語の重要度を評価する指標
tf:単語の出現頻度
idf:単語の文書内における希少度
"""
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

def make_features(train,valid,test):
    """
    train,valid,test:それぞれ前処理済のdataframe
    name：出力名
    """
    df = pd.concat([train,valid,test],axis=0)#3つのdataframeを連結
    #↑連結しておかないと使用している特徴量が一致しないので、入力の形式のエラーになる．
    wc = CountVectorizer()#出現する単語のカウントを特徴量とする
    x = wc.fit_transform(df["TITLE"])
    features = np.array(x.toarray())#numpy.ndarrayに変換
    df_features = pd.DataFrame(features, columns=wc.get_feature_names_out())#get_feature_names_out:特徴量ラベル

    #dataframeを3つに分けてreturnする
    train_feature = df_features[:10672]#〜train
    valid_feature = df_features[10672:12006]#train終わり〜valid
    test_feature = df_features[12006:]#valid終わり〜

    return train_feature.to_csv("train.feature.txt",sep="\t",index=False),\
    valid_feature.to_csv("valid.feature.txt",sep="\t",index=False),\
    test_feature.to_csv("test.feature.txt",sep="\t",index=False)

In [38]:
#学習データ，検証データ，評価データそれぞれの特徴量を抽出し出力する
make_features(train_prep,valid_prep,test_prep)

(None, None, None)